In [1]:
# Karthik Unnikrishnan
# Sai Kumar Kayala

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, basic, item_knn, user_knn, als, funksvd
from lenskit import topn
from lenskit.metrics import predict


In [3]:
ratings=pd.read_csv('ml-1m/ratings.dat', sep='::',
                      names=['user', 'item', 'rating', 'timestamp'])

/home/sai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:

from lenskit.algorithms import Predictor
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

def eval( algo, train, test):
    algo = util.clone(algo)
    algo.fit(train)
    preds = Predictor.predict(algo, test)
    return preds

In [5]:
# for FunkSVD as base model to compare other approaches

algo_funk=funksvd.FunkSVD(4, iterations=1000, lrate=0.0002)
truth=[]
preds_funk=[]

for train, test in xf.partition_users(ratings, 5, xf.SampleFrac(0.2)):
    truth.append(test)
    preds_funk.append(eval( algo_funk, train, test))


truth=pd.concat(truth,ignore_index=True)
preds_funk=pd.concat(preds_funk,ignore_index=True)    
    
rmse_funk=predict.rmse(truth['rating'],preds_funk.values,missing='ignore')

In [6]:
print("RMSE value for FunkSVD is:",str(rmse_funk))

RMSE value for FunkSVD is: 0.8675086191649507


In [7]:
#import libraries for NN approach
import keras
from keras.layers import Embedding, Reshape, dot
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras import backend as K
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [8]:
#Get number of unique users and ratings
n_users, n_movies = max(ratings.user.unique()), max(ratings.item.unique())

In [9]:
ratings.userId = ratings.user.astype('category').cat.codes.values
ratings.movieId = ratings.item.astype('category').cat.codes.values

/home/sai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/home/sai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [10]:
#Split the data into training set (95%) and test set(5%)
train, test = train_test_split(ratings, test_size=0.05)

In [11]:
#RMSE loss function for optimization

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [12]:
#define the first model 
#Model that simply takes the sdot product of the user and item latent factor embeddings to predict scores

def embedding_v1(n_latent_factors) :
    movie_input = keras.layers.Input(shape=[1],name='Item')
    movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
    movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

    user_input = keras.layers.Input(shape=[1],name='User')
    user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))
    prod = keras.layers.dot([movie_vec, user_vec], axes=1,name='DotProduct')
    
    model = keras.Model([user_input, movie_input], prod)
    model.compile('adam', loss = root_mean_squared_error, 
              metrics =["accuracy"])
    
    return model

In [13]:
# Initialize model with 10 latent features
model_1 = embedding_v1(5)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
#Get a summary of model
model_1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 5)         19765       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         30205       User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [15]:
#fit the model for training data
history = model_1.fit([train.user, train.item], train.rating, epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
950198/950198 [==============================] - 31s 33us/step - loss: 1.6728 - acc: 0.2786
Epoch 2/10
950198/950198 [==============================] - 30s 31us/step - loss: 0.9379 - acc: 0.4153
Epoch 3/10
950198/950198 [==============================] - 30s 31us/step - loss: 0.9232 - acc: 0.4200
Epoch 4/10
950198/950198 [==============================] - 30s 31us/step - loss: 0.9185 - acc: 0.4214
Epoch 5/10
950198/950198 [==============================] - 30s 31us/step - loss: 0.9115 - acc: 0.4246
Epoch 6/10
950198/950198 [==============================] - 30s 32us/step - loss: 0.8995 - acc: 0.4312
Epoch 7/10
950198/950198 [==============================] - 30s 31us/step - loss: 0.8870 - acc: 0.4387
Epoch 8/10
950198/950198 [==============================] - 30s 32us/step - loss: 0.8783 - acc: 0.4440
Epoch 9/10
950198/950198 [==============================] - 30s 32us/step - loss: 0.8718 - acc: 0.4473
Epoch 10/10
950198/950198

In [16]:
y_pred=model_1.predict([test.user,test.item])

In [43]:
rms= model_1.evaluate([test.user, test.item], test.rating)[0]
print("RMSE for model_1 is:"+str(rms))

50011/50011 [==============================] - 1s 15us/step
RMSE for model_1 is:0.8741118925519887


In [55]:
# Model 2: Adding 1 dimensional convolutional layers to the concatenated user and movie embeddings, 
# and then feeding it to a fully connected network

def embeddings_conv1D(n_latent_factors_user, n_latent_factors_movie):
    
    movie_input = keras.layers.Input(shape=[1],name='Item')
    movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
    movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

    user_input = keras.layers.Input(shape=[1],name='User')
    user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))

    concat = keras.layers.Concatenate()([movie_vec, user_vec])
    dense = keras.layers.Dense(256,name='FullyConnected')(concat)
    dense=keras.layers.Reshape((256,1))(dense)
    dense=keras.layers.Conv1D(128,3)(dense)
    dense=keras.layers.Conv1D(5,3)(dense)
    dense=keras.layers.Flatten()(dense)
    dense_3 = keras.layers.Dense(64,name='FullyConnected-2')(dense)
    dense_4 = keras.layers.Dense(10,name='FullyConnected-3', activation='relu')(dense_3)


    result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
    adam = Adam(lr=0.001)
    model = keras.Model([user_input, movie_input], result)
    model.compile(optimizer=adam,loss= root_mean_squared_error)
    return model

In [56]:
model_2=embeddings_conv1D(5,5)

In [57]:
model_2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 5)         19765       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         30205       User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [58]:
history = model_2.fit([train.user, train.item], train.rating, epochs=10,batch_size=128)

Epoch 1/10
950198/950198 [==============================] - 34s 36us/step - loss: 0.9360
Epoch 2/10
950198/950198 [==============================] - 34s 36us/step - loss: 0.9050
Epoch 3/10
950198/950198 [==============================] - 38s 40us/step - loss: 0.8932
Epoch 4/10
950198/950198 [==============================] - 42s 44us/step - loss: 0.8859
Epoch 5/10
950198/950198 [==============================] - 38s 40us/step - loss: 0.8809
Epoch 6/10
950198/950198 [==============================] - 40s 42us/step - loss: 0.8759
Epoch 7/10
950198/950198 [==============================] - 38s 40us/step - loss: 0.8716
Epoch 8/10
950198/950198 [==============================] - 32s 34us/step - loss: 0.8681
Epoch 9/10
950198/950198 [==============================] - 37s 39us/step - loss: 0.8652
Epoch 10/10
950198/950198 [==============================] - 34s 36us/step - loss: 0.8627


In [63]:
rms= model_2.evaluate([test.user, test.item], test.rating)
print(rms)

50011/50011 [==============================] - 1s 27us/step
0.8701149790639466


In [30]:
# Model with embeddings and fully connected layers

def embedding_fully_connected(n_latent_factors_user, n_latent_factors_movie):
    
    movie_input = keras.layers.Input(shape=[1],name='Item')
    movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
    movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

    user_input = keras.layers.Input(shape=[1],name='User')
    user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))

    concat = keras.layers.Concatenate()([movie_vec, user_vec])
    # add fully-connected-layers
    fc1 = keras.layers.Dense(128, activation='relu')(concat)
    fc2 = keras.layers.Dense(32, activation='relu')(fc1)

    result = keras.layers.Dense(1, activation='linear',name='Activation')(fc2)
    adam = Adam(lr=0.001)
    model = keras.Model([user_input, movie_input], result)
    model.compile(optimizer=adam,loss= root_mean_squared_error)
    return model

In [32]:
model_3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 5)         19765       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         30205       User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [31]:
model_3=embedding_fully_connected(5,5)

In [64]:
history = model_3.fit([train.user, train.item], train.rating, epochs=10)

Epoch 1/10
950198/950198 [==============================] - 46s 48us/step - loss: 0.8315
Epoch 2/10
950198/950198 [==============================] - 45s 47us/step - loss: 0.8296
Epoch 3/10
950198/950198 [==============================] - 46s 49us/step - loss: 0.8283
Epoch 4/10
950198/950198 [==============================] - 47s 49us/step - loss: 0.8270
Epoch 5/10
950198/950198 [==============================] - 46s 48us/step - loss: 0.8258
Epoch 6/10
950198/950198 [==============================] - 45s 48us/step - loss: 0.8249
Epoch 7/10
950198/950198 [==============================] - 47s 49us/step - loss: 0.8237
Epoch 8/10
950198/950198 [==============================] - 47s 49us/step - loss: 0.8229
Epoch 9/10
950198/950198 [==============================] - 46s 49us/step - loss: 0.8221
Epoch 10/10
950198/950198 [==============================] - 47s 49us/step - loss: 0.8214


In [65]:
rms= model_3.evaluate([test.user, test.item], test.rating)
print("RMSE for model_3 is:"+str(rms))

50011/50011 [==============================] - 1s 16us/step
RMSE for model_2 is:0.8514534224500582


Conclusions:

As the baseline, we tried FunkSVD, which gave an RMSE of around 0.87

For the neural network approach, we tried the following configurations:

Model 1: This model used embeddings of user and items to 5 latent factors each. Then, a doot product of these embeddings was taken to get a predicted score. The RMSE in this case was around 0.88, which is very close to what we got for FunkSVD

Model 2:  For this we tried adding 1D convolution layers after concatenating the user and movie embeddings. There was a slight improvement in RMSE though the improvement seemed to stagnate after 5 epochs.

Model 2: In this model we added multiple fully connected layers, with 126 and 32 nodes each. We set the latent factors to 5 for users, and 5 for movies. This model gives an RMSE value of 0.85, which is better that all our previous models and the FunkSVD baseline









Reference: For model 1: https://github.com/Gurupradeep/Movie-Recommendation-System/blob/master/MovieLens_Recommendation_Notebook.ipynb